In [1]:
import pandas as pd
import requests
import os

from datetime import date
from time import sleep
from tqdm import tqdm_notebook

In [2]:
'''
Import raw data downloaded from bipro.prozorro interface
Later it will be used as a source of proper contract ids for downloading data from public API
'''

data_raw = pd.read_excel('02_data/Ostap_contracts_12.11.2018_v0.2.xls', 'Sheet1')
data_gas_all = data_raw.loc[data_raw['Лот'].str.contains('природн', case = False)]
print(data_gas_all.shape)
contracts_gas = data_gas_all['IDContractOrig'].unique()
print(len(contracts_gas))
data_gas_all.head()

(18562, 10)
4407


,Идентификатор,ИдентификаторЛота,IDOrganizator,IDContractOrig,Дата,Дата підписання договору,Тип методу закупівлі,Класифікація CPV лота,Лот,Дата внесення змін до договору
0,UA-2017-05-15-002292-b,UA-2017-05-15-002292-b-L1,34716922,244af7a6ee364ed3a8b3cff3cedabc3f,2017-05-15,2017-08-01,Відкриті торги,09120000-6 Газове паливо,Природний газ для потреб Львівського навчально...,2017-10-11
1,UA-2017-06-21-000460-b,UA-2017-06-21-000460-b-L1,8294952,28cf7ff64b564839b13c8c91c98676dc,2017-06-21,2017-08-02,Відкриті торги,09120000-6 Газове паливо,Природний газ,2017-08-29
2,UA-2017-06-21-000460-b,UA-2017-06-21-000460-b-L1,8294952,28cf7ff64b564839b13c8c91c98676dc,2017-06-21,2017-08-02,Відкриті торги,09120000-6 Газове паливо,Природний газ,2017-09-20
3,UA-2017-06-21-000460-b,UA-2017-06-21-000460-b-L1,8294952,28cf7ff64b564839b13c8c91c98676dc,2017-06-21,2017-08-02,Відкриті торги,09120000-6 Газове паливо,Природний газ,2017-09-25
4,UA-2017-06-21-000460-b,UA-2017-06-21-000460-b-L1,8294952,28cf7ff64b564839b13c8c91c98676dc,2017-06-21,2017-08-02,Відкриті торги,09120000-6 Газове паливо,Природний газ,2017-10-10


In [3]:
'''
Functions for downloading all attached docs, mainly contracts and additional agreements
'''

dir_base = '00_contracts_pdf'
error_ids = []

'''
Sometimes contract authorities upload files with identical filenames, so we need a separate function to fix this problem
'''
def set_unique_titles(docs):
    titles = [doc['title'].replace("..",".") for doc in docs]
    titles_unique = []
    for title in titles:
        if not title in titles_unique:
            titles_unique.append(title)
        else:
            name = title
            name_splitted = name.split('.')
            name_format = name_splitted[-1]
            count = 2
            while name in titles_unique:
                name = name.replace('(%i)' % (count - 1), '')
                name = name[:len(name_format)*-1-1] + '(%i).' % count + name_format
                count += 1
            titles_unique.append(name)
    return titles_unique

'''
Download all the docs, attached to the respective contract
In case files are already downloaded, function skips them and produces relevant message
Sometimes files are not attached to the contract id - these cases are stored separately
'''
def download_docs(contract_id,date,response_data):
    
    current_dir = os.path.join(dir_base,str(date),contract_id)

    if not os.path.exists(current_dir):
        os.makedirs(current_dir)
        
    try:
        docs = response_data['documents']
        titles_unique = set_unique_titles(docs)
        for i, doc in tqdm_notebook(enumerate(docs)):
            if doc['format'] != 'application/pkcs7-signature':
                title = titles_unique[i].replace('/','-')
                filepath = os.path.join(current_dir,title)
                if not os.path.exists(filepath):
                    response_file = requests.get(doc['url'])
                    try:
                        with open(filepath, 'wb') as f:
                            f.write(response_file.content)
                        print("Збережено файл " + filepath)
                    except FileNotFoundError:
                        print("Неможливо зберегти " + filepath)
                    sleep(0.1)
                else:
                    print("Файл %s уже існує" %filepath)
    except KeyError:
        error_ids.append(id_api)
        print('Для id %s відсутні прикріплені документи. Додано до переліку error_ids' %id_api)


In [4]:
'''
Functions for creating lists of relevant data from the responses from ProZorro API
'''

'''
Process data obtained from contracts entrypoint of API
Generates a list for every additional agreement, including both general info on the main contract and specific info on the agreement
'''
def process_response_data(response_data,tender_data):
    records_contract = []

    contract_id = response_data['id']
    tender_id = response_data['tender_id']
    date_signed = pd.to_datetime(response_data['dateSigned'][:10])
    
    try:
        contract_start = response_data['period']['startDate'][:10]
    except KeyError:
        contract_start = 'not specified'
        print("Відсутня дата початку дії договору")
    try:
        contract_end = response_data['period']['endDate'][:10]
    except KeyError:
        contract_end = 'not specified'
        print("Відсутня дата кінця дії договору")

    if response_data['value']['valueAddedTaxIncluded']:
        contract_value = response_data['value']['amount']
    elif not response_data['value']['valueAddedTaxIncluded']:
        contract_value = response_data['value']['amount']*1.2

    procuring_entity_name = response_data['procuringEntity']['identifier']['legalName']
    procuring_entity_id = response_data['procuringEntity']['identifier']['id']
    procuring_entity_type = response_data['procuringEntity']['kind']
    
    try:
        region = response_data['procuringEntity']['address']['region']
    except KeyError:
        region = 'not specified'
        print("Відсутній регіон")
    
    quantity = sum([item['quantity'] for item in response_data['items']])
    units = response_data['items'][0]['unit']['code']
    item_desc = response_data['items'][0]['description']
    cpv = response_data['items'][0]['classification']['id']

    for change in response_data['changes']:
        rationale_type = ', '.join(str(rationaleType) for rationaleType in change['rationaleTypes'])
        rationale = change['rationale']
        try:
            date_addcontr = pd.to_datetime(change['dateSigned'][:10]).date()
        except KeyError:
            date_addcontr = 'unknown'
        records_contract.append([contract_id,
                                 date_signed,
                                 procuring_entity_name,
                                 procuring_entity_id,
                                 procuring_entity_type,
                                 region,
                                 contract_value,
                                 quantity,
                                 units,
                                 item_desc,
                                 cpv,
                                 contract_start,
                                 contract_end,
                                 date_addcontr,
                                 rationale_type,
                                 rationale] + tender_data)
        
    return records_contract

'''
Process data obtained from tenders entrypoint of API
Produces only general data for each tender, which is later fed to process_response_data()
'''
def get_tender_data(tender_id):
    entry_point_tenders = 'https://public.api.openprocurement.org/api/2.4/tenders/'
    response_data_tenders = requests.get(entry_point_tenders + tender_id).json()['data']
    tender_id = response_data_tenders['tenderID']
    procurement_type = response_data_tenders['procurementMethodType']
    sleep(0.1)
    return [tender_id,procurement_type]
    

In [5]:
'''
This chunk of code contains the main cycle, which initiates functions for downloading docs and creating lists 
of relevant data for each contract id
Docs are downloaded only in case contract value exceeds 1 mln UAH
'''

'''
the next line should be uncommented before the first run in session and later commented again.
otherwise, the process of getting data will be substantially slowed down
because of the need to get the data, which has already been processed
'''
#records, recorded_ids = [], []

print("Записано %i договорів" % len(recorded_ids))
double_check = []

entry_point_contracts = 'https://public.api.openprocurement.org/api/2.4/contracts/'

'''
For educational purposes, only the first ten contracts are processed
In order to fix this, simply remove [:10]
'''
for contract_id in tqdm_notebook(contracts_gas[:10]):
    if not contract_id in recorded_ids:
        response_data = requests.get(entry_point_contracts + contract_id).json()['data']
        tender_data = get_tender_data(response_data['tender_id'])
        records_contract = process_response_data(response_data,tender_data)
        if records_contract[0][6] > 10**6:
            download_docs(contract_id,records_contract[0][1].date(),response_data)
        recorded_ids.append(contract_id)
        records = records + records_contract
        sleep(0.1)
    else:
        double_check.append(contract_id)
        
print("Перевірка кількості вже записаних договорів: " + str(len(double_check)))

Записано 0 договорів


Збережено файл 00_contracts_pdf\2017-08-01\244af7a6ee364ed3a8b3cff3cedabc3f\Договір.pdf
Збережено файл 00_contracts_pdf\2017-08-01\244af7a6ee364ed3a8b3cff3cedabc3f\Додаткова угода.pdf


Збережено файл 00_contracts_pdf\2017-08-02\28cf7ff64b564839b13c8c91c98676dc\Договір №С-257-17 від 02.08.2017
Збережено файл 00_contracts_pdf\2017-08-02\28cf7ff64b564839b13c8c91c98676dc\Додаткова угода №1 від 29.08.2017 року
Збережено файл 00_contracts_pdf\2017-08-02\28cf7ff64b564839b13c8c91c98676dc\ДОП УГОДА №2 20 .09.pdf
Збережено файл 00_contracts_pdf\2017-08-02\28cf7ff64b564839b13c8c91c98676dc\доп.угода. 3 25.09.PDF
Збережено файл 00_contracts_pdf\2017-08-02\28cf7ff64b564839b13c8c91c98676dc\доп угода 4  10.10.17.pdf
Збережено файл 00_contracts_pdf\2017-08-02\28cf7ff64b564839b13c8c91c98676dc\доп уг. 5  03.11.17.pdf
Збережено файл 00_contracts_pdf\2017-08-02\28cf7ff64b564839b13c8c91c98676dc\Доп.угода 6 ГАЗ.pdf
Збережено файл 00_contracts_pdf\2017-08-02\28cf7ff64b564839b13c8c91c98676dc\доп угода 7 ГАЗ.pdf


Збережено файл 00_contracts_pdf\2017-08-07\a48b5748dff4497ea2bf4e850b45d768\Договір по Газу.pdf
Збережено файл 00_contracts_pdf\2017-08-07\a48b5748dff4497ea2bf4e850b45d768\дод.угода газ 21.09.17.pdf
Збережено файл 00_contracts_pdf\2017-08-07\a48b5748dff4497ea2bf4e850b45d768\Додаткова угода №2 Газ 04.10.2017р.pdf
Збережено файл 00_contracts_pdf\2017-08-07\a48b5748dff4497ea2bf4e850b45d768\Додаткова угода №3.jpg
Збережено файл 00_contracts_pdf\2017-08-07\a48b5748dff4497ea2bf4e850b45d768\Додаткова угода №3(2).jpg
Збережено файл 00_contracts_pdf\2017-08-07\a48b5748dff4497ea2bf4e850b45d768\Додаткова угода №4.pdf
Збережено файл 00_contracts_pdf\2017-08-07\a48b5748dff4497ea2bf4e850b45d768\Додаткова угода №4.jpg
Збережено файл 00_contracts_pdf\2017-08-07\a48b5748dff4497ea2bf4e850b45d768\Додаткова угода №5.pdf
Збережено файл 00_contracts_pdf\2017-08-07\a48b5748dff4497ea2bf4e850b45d768\Додаткова угода №5.jpeg
Збережено файл 00_contracts_pdf\2017-08-07\a48b5748dff4497ea2bf4e850b45d768\дод.угода га

Збережено файл 00_contracts_pdf\2017-08-08\77779da5daae470487be737adcb293b7\Договір №24-17 ГК ТОВ ГК ІНВЕСТСЕРВІС від 08.08.17р.pdf
Збережено файл 00_contracts_pdf\2017-08-08\77779da5daae470487be737adcb293b7\ДУ ТОВ ГК Інвестсервіс.pdf
Збережено файл 00_contracts_pdf\2017-08-08\77779da5daae470487be737adcb293b7\Додаткова угода №2 Інвестсервіс.pdf
Збережено файл 00_contracts_pdf\2017-08-08\77779da5daae470487be737adcb293b7\ДУ Інвестсервіс 20.11.17.pdf
Збережено файл 00_contracts_pdf\2017-08-08\77779da5daae470487be737adcb293b7\ДУ №4 Інвестсервіс.pdf


Збережено файл 00_contracts_pdf\2017-08-15\a5708bd3a60d4f9a909d1100d9015c67\договір №170000191 від 15.08.2017 року з ДП Укравтогаз.pdf
Збережено файл 00_contracts_pdf\2017-08-15\a5708bd3a60d4f9a909d1100d9015c67\Додаткова угода до договору 1708000191 ДП Укравтогаз.pdf
Збережено файл 00_contracts_pdf\2017-08-15\a5708bd3a60d4f9a909d1100d9015c67\повідомлення про внесення змін до договору ДП Укравтогаз від 04.09.2017р.pdf

Перевірка кількості вже записаних договорів: 0


In [8]:
'''
Create pandas dataframe from previously created list of records
Also transforming some columns to proper datetime format and ommiting some data concerning 
purchasing of automobile gas and technological needs
'''

columns = ['contract_id','date_signed','procuring_entity','procuring_entity_id','procuring_entity_type','region','contract_value','quantity','units','item_desc','cpv','contract_start','contract_end','addcontr_date','rationale_type','rationale','tender_id','procurement_type']
data_addcontr_raw = pd.DataFrame.from_records(records, columns = columns)
print(data_addcontr_raw.shape)
print(len(data_addcontr_raw['contract_id'].unique()))

data_addcontr_raw['date_signed'] = data_addcontr_raw.loc[:,'date_signed'].map(lambda x: x.date())
data_addcontr_raw['contract_start'] = data_addcontr_raw.loc[:,'contract_start'].map(lambda x: pd.to_datetime(x).date() if x != 'not specified' else x)
data_addcontr_raw['contract_end'] = data_addcontr_raw.loc[:,'contract_end'].map(lambda x: pd.to_datetime(x).date() if x != 'not specified' else x)
data_addcontr_raw['tenderURL'] = data_addcontr_raw.loc[:,'tender_id'].map(lambda x: 'https://prozorro.gov.ua/tender/'+ x)

data_addcontr_raw = data_addcontr_raw.loc[~data_addcontr_raw['item_desc'].str.contains('заправк', case = False)]
data_addcontr_raw = data_addcontr_raw.loc[~data_addcontr_raw['item_desc'].str.contains('забезпечення виробничо-технологічних потреб', case = False)]

print(data_addcontr_raw.shape)
print(len(data_addcontr_raw['contract_id'].unique()))

data_addcontr_raw.head(3)

(24553, 18)
4407
(24524, 19)
4394


,contract_id,date_signed,procuring_entity,procuring_entity_id,procuring_entity_type,region,contract_value,quantity,units,item_desc,cpv,contract_start,contract_end,addcontr_date,rationale_type,rationale,tender_id,procurement_type,tenderURL
0,244af7a6ee364ed3a8b3cff3cedabc3f,2017-08-01,"Державний вищий навчальний заклад ""Університет...",34716922,general,Київська область,1093519.824,162450,MTQ,Природний газ для потреб Львівського навчально...,09120000-6,2017-08-01,2017-12-31,2017-10-11,"volumeCuts, itemPriceVariation",Згідно довідки Торгово-промислової палати Укра...,UA-2017-05-15-002292-b,aboveThresholdUA,https://prozorro.gov.ua/tender/UA-2017-05-15-0...
1,28cf7ff64b564839b13c8c91c98676dc,2017-08-02,Квартирно - експлуатаційний відділ м. Херсона,08294952,general,Херсонська область,1124468.370,202501,MTQ,Природний газ,09120000-6,2017-08-02,2017-12-31,2017-08-29,volumeCuts,Зменшення обсягів закупівлі з метою приведення...,UA-2017-06-21-000460-b,aboveThresholdUA,https://prozorro.gov.ua/tender/UA-2017-06-21-0...
2,28cf7ff64b564839b13c8c91c98676dc,2017-08-02,Квартирно - експлуатаційний відділ м. Херсона,08294952,general,Херсонська область,1124468.370,202501,MTQ,Природний газ,09120000-6,2017-08-02,2017-12-31,2017-09-20,itemPriceVariation,Підвищення вартості природного газу на ринку У...,UA-2017-06-21-000460-b,aboveThresholdUA,https://prozorro.gov.ua/tender/UA-2017-06-21-0...


In [9]:
'''
Exporting data to xlsx
'''
data_addcontr_raw.to_excel('02_data/00_data_addcontr_all_v5.xlsx', index = False)